# Assignment 1

## 1.0 Import and install python libraries

In [2]:
# import numpy and pandas libraries
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier 
import summarytools
from summarytools import dfSummary
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
# set random seed to ensure that results are repeatable
np.random.seed(1)

## 3.0 Load data 

In [3]:
X_train = pd.read_csv("./data/Rscores_X_train.csv")
X_test = pd.read_csv("./data/Rscores_X_test.csv")
y_train = pd.read_csv("./data/Rscores_y_train.csv")
y_test = pd.read_csv("./data/Rscores_y_test.csv")

In [4]:
performance = pd.DataFrame({"model": [], "Accuracy": [], "Precision": [], "Recall": [], "F1": []})

# Logistic Regression

In [5]:
log_reg_model = LogisticRegression(penalty='none',max_iter=2000)
_ = log_reg_model.fit(X_train, np.ravel(y_train,order='C'))

c:\Users\mukes\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [6]:
model_preds = log_reg_model.predict(X_test)
c_matrix = confusion_matrix(y_test, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"default logistic", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,default logistic,0.826087,0.839286,0.979167,0.903846


# Randome search - Logistic

In [7]:
#random search in log
score_measure = "precision_macro"
kfolds = 15

param_grid = {
    'C':[0.1,1,10,100],
    'penalty': ['l2'],
    'solver': ['liblinear']
    
}

logreg_rand = LogisticRegression()

rand_search_log = RandomizedSearchCV(estimator = logreg_rand, param_distributions=param_grid, cv=kfolds, n_iter=4,
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)

_ = rand_search_log.fit(X_train,np.ravel(y_train,order='C'))

print(f"The best {score_measure} score is {rand_search_log.best_score_}")
print(f"... with parameters: {rand_search_log.best_params_}")

bestRecallTree = rand_search_log.best_estimator_

Fitting 15 folds for each of 4 candidates, totalling 60 fits
The best precision_macro score is 0.3455509705431804
... with parameters: {'solver': 'liblinear', 'penalty': 'l2', 'C': 100}


In [8]:
model_preds = rand_search_log.predict(X_test)
c_matrix = confusion_matrix(y_test, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"rand_search_log_l2", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,default logistic,0.826087,0.839286,0.979167,0.903846
0,rand_search_log_l2,0.827586,0.840708,0.979381,0.904762


# Grid search - Logistic

In [9]:
score_measure = "precision_macro"
kfolds = 5

param_grid = {
    'C': [0.1, 1, 10, 100],
    'penalty': ['l1'],
    'solver': ['liblinear']
}

logreg_grid = LogisticRegression()
grid_search_log = GridSearchCV(logreg_grid, param_grid=param_grid, cv=kfolds, 
                               scoring=score_measure, verbose=1, n_jobs=-1, 
                               return_train_score=True, error_score='raise')

_ = grid_search_log.fit(X_train, np.ravel(y_train))

print(f"The best macro-averaged precision score is {grid_search_log.best_score_}")
print(f"... with parameters: {grid_search_log.best_params_}")

bestLogReg = grid_search_log.best_estimator_

Fitting 5 folds for each of 4 candidates, totalling 20 fits
The best macro-averaged precision score is 0.36766437367434
... with parameters: {'C': 10, 'penalty': 'l1', 'solver': 'liblinear'}


In [10]:
model_preds = grid_search_log.predict(X_test)
c_matrix = confusion_matrix(y_test, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"grid_search_log_l1", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,default logistic,0.826087,0.839286,0.979167,0.903846
0,rand_search_log_l2,0.827586,0.840708,0.979381,0.904762
0,grid_search_log_l1,0.827586,0.840708,0.979381,0.904762


# SVM

In [11]:
#svm with linear kernel
svm_lin_model = SVC(kernel="linear",probability=True)
_ = svm_lin_model.fit(X_train, np.ravel(y_train))

In [12]:
model_preds = svm_lin_model.predict(X_test)
c_matrix = confusion_matrix(y_test, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"lin svm", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,default logistic,0.826087,0.839286,0.979167,0.903846
0,rand_search_log_l2,0.827586,0.840708,0.979381,0.904762
0,grid_search_log_l1,0.827586,0.840708,0.979381,0.904762
0,lin svm,0.797980,0.814433,0.975309,0.887640


# SVM with random search

In [30]:
#random search in SVM
from scipy.stats import uniform
score_measure = "precision_macro"
kfolds = 5

param_grid = {
    'kernel':['linear'],
    'C': uniform(loc=0, scale=4),
    'gamma': uniform(loc=0, scale=1)
    
}

SVM_R_out = SVC()
rand_search = RandomizedSearchCV(estimator = SVM_R_out, param_distributions=param_grid, cv=kfolds, n_iter=4,
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)

_ = rand_search.fit(X_train,np.ravel(y_train))

print(f"The best {score_measure} score is {rand_search.best_score_}")
print(f"... with parameters: {rand_search.best_params_}")

bestRecallTree = rand_search.best_estimator_

Fitting 5 folds for each of 4 candidates, totalling 20 fits
The best precision_macro score is 0.40889779720361014
... with parameters: {'C': 0.11594511156940346, 'gamma': 0.8027627416090345, 'kernel': 'linear'}


In [31]:
model_preds = rand_search.predict(X_test)
c_matrix = confusion_matrix(y_test, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"rand_poly_svm", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,default logistic,0.826087,0.839286,0.979167,0.903846
0,rand_search_log_l2,0.827586,0.840708,0.979381,0.904762
0,grid_search_log_l1,0.827586,0.840708,0.979381,0.904762
0,lin svm,0.797980,0.814433,0.975309,0.887640
0,dtree,0.994595,0.993056,1.000000,0.996516
0,rand_poly_tree,0.989189,0.993007,0.993007,0.993007
0,grid_poly_dtree,0.994565,0.993007,1.000000,0.996491
0,"NN-60,50,40",0.994565,0.993007,1.000000,0.996491
0,NN-random,0.994565,0.993007,1.000000,0.996491
0,NN-grid,0.994565,0.993007,1.000000,0.996491


# SVM Grid search

In [82]:
#grid search in SVM
score_measure = "precision_macro"
kfolds = 5

param_grid = {
    'C':[0.1,1,10,100],
    'gamma':[1,0.1,0.01,0.001],
    'kernel':['poly']
    
}

SVM_G_out = SVC()
grid_search = GridSearchCV(estimator = SVM_G_out, param_grid=param_grid, cv=kfolds, 
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)

_ = grid_search.fit(X_train,np.ravel(y_train))

print(f"The best {score_measure} score is {grid_search.best_score_}")
print(f"... with parameters: {grid_search.best_params_}")

bestRecallTree = grid_search.best_estimator_

Fitting 5 folds for each of 16 candidates, totalling 80 fits


In [ ]:
model_preds = grid_search.predict(X_test)
c_matrix = confusion_matrix(y_test, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"grid_poly_svm", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

# Dtree

In [13]:
dtree=DecisionTreeClassifier()
_ = dtree.fit(X_train, y_train)

In [14]:
model_preds = dtree.predict(X_test)
c_matrix = confusion_matrix(y_test, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"dtree", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,default logistic,0.826087,0.839286,0.979167,0.903846
0,rand_search_log_l2,0.827586,0.840708,0.979381,0.904762
0,grid_search_log_l1,0.827586,0.840708,0.979381,0.904762
0,lin svm,0.797980,0.814433,0.975309,0.887640
0,dtree,0.994595,0.993056,1.000000,0.996516


# DTree Random search

In [15]:
#random with tree
score_measure = "precision_macro"
kfolds = 5

param_grid = {
    'min_samples_split': np.arange(2,42),  
    'min_samples_leaf': np.arange(2,42),
    'max_leaf_nodes': np.arange(5, 42), 
    'max_depth': np.arange(2,42), 
    'criterion': ['entropy', 'gini'],
}

dtree = DecisionTreeClassifier()
rand_search_tree = RandomizedSearchCV(estimator = dtree, param_distributions=param_grid, cv=kfolds, n_iter=500,
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)
 
_ = rand_search_tree.fit(X_train, y_train)

print(f"The best {score_measure} score is {rand_search_tree.best_score_}")
print(f"... with parameters: {rand_search_tree.best_params_}")

bestRecallTree = rand_search_tree.best_estimator_

Fitting 5 folds for each of 500 candidates, totalling 2500 fits
The best precision_macro score is 0.9733774638589769
... with parameters: {'min_samples_split': 9, 'min_samples_leaf': 4, 'max_leaf_nodes': 32, 'max_depth': 17, 'criterion': 'entropy'}


In [16]:
model_preds = rand_search_tree.predict(X_test)
c_matrix = confusion_matrix(y_test, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"rand_poly_tree", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,default logistic,0.826087,0.839286,0.979167,0.903846
0,rand_search_log_l2,0.827586,0.840708,0.979381,0.904762
0,grid_search_log_l1,0.827586,0.840708,0.979381,0.904762
0,lin svm,0.797980,0.814433,0.975309,0.887640
0,dtree,0.994595,0.993056,1.000000,0.996516
0,rand_poly_tree,0.989189,0.993007,0.993007,0.993007


# Dtree with Grid

In [17]:
#grid with tree
score_measure = "precision_macro"
kfolds = 5

param_grid = {
    'min_samples_split': np.arange(3,6),  
    'min_samples_leaf': np.arange(3,6),
    'min_impurity_decrease': np.arange(0.0009, 0.0012,0.0001),
    'max_leaf_nodes': np.arange(36,40), 
    'max_depth': np.arange(39,42), 
    'criterion': ['entropy'],
}

dtree = DecisionTreeClassifier()
grid_search_tree = GridSearchCV(estimator = dtree, param_grid=param_grid, cv=kfolds, 
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)

_ = grid_search_tree.fit(X_train, y_train)

print(f"The best {score_measure} score is {grid_search_tree.best_score_}")
print(f"... with parameters: {grid_search_tree.best_params_}")

bestRecallTree = grid_search_tree.best_estimator_

Fitting 5 folds for each of 324 candidates, totalling 1620 fits
The best precision_macro score is 0.9753981515046645
... with parameters: {'criterion': 'entropy', 'max_depth': 39, 'max_leaf_nodes': 38, 'min_impurity_decrease': 0.0009, 'min_samples_leaf': 3, 'min_samples_split': 3}


In [18]:
model_preds = grid_search_tree.predict(X_test)
c_matrix = confusion_matrix(y_test, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"grid_poly_dtree", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,default logistic,0.826087,0.839286,0.979167,0.903846
0,rand_search_log_l2,0.827586,0.840708,0.979381,0.904762
0,grid_search_log_l1,0.827586,0.840708,0.979381,0.904762
0,lin svm,0.797980,0.814433,0.975309,0.887640
0,dtree,0.994595,0.993056,1.000000,0.996516
0,rand_poly_tree,0.989189,0.993007,0.993007,0.993007
0,grid_poly_dtree,0.994565,0.993007,1.000000,0.996491


In [19]:
from __future__ import print_function
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
import matplotlib.pyplot as plt

from sklearn import datasets

In [27]:
%%time

ann = MLPClassifier(hidden_layer_sizes=(100,), activation='relu',solver='adam', max_iter=200)
_ = ann.fit(X_train, np.ravel(y_train))

Wall time: 212 ms


In [21]:
%%time
y_pred = ann.predict(X_test)

Wall time: 886 µs


In [28]:
model_preds = grid_search_tree.predict(X_test)
c_matrix = confusion_matrix(y_test, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"NN-100", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,default logistic,0.826087,0.839286,0.979167,0.903846
0,rand_search_log_l2,0.827586,0.840708,0.979381,0.904762
0,grid_search_log_l1,0.827586,0.840708,0.979381,0.904762
0,lin svm,0.797980,0.814433,0.975309,0.887640
0,dtree,0.994595,0.993056,1.000000,0.996516
0,rand_poly_tree,0.989189,0.993007,0.993007,0.993007
0,grid_poly_dtree,0.994565,0.993007,1.000000,0.996491
0,"NN-60,50,40",0.994565,0.993007,1.000000,0.996491
0,NN-random,0.994565,0.993007,1.000000,0.996491
0,NN-grid,0.994565,0.993007,1.000000,0.996491


In [23]:
%%time

score_measure = "accuracy"
kfolds = 5

param_grid = {
    'hidden_layer_sizes': [ (50,), (70,),(50,30), (40,20), (60,40, 20), (70,50,40)],
    'activation': ['logistic', 'tanh', 'relu'],
    'solver': ['adam', 'sgd'],
    'alpha': [0, .2, .5, .7, 1],
    'learning_rate': ['constant', 'invscaling', 'adaptive'],
    'learning_rate_init': [0.001, 0.01, 0.1, 0.2, 0.5],
    'max_iter': [5000]
}

ann = MLPClassifier()
grid_search = RandomizedSearchCV(estimator = ann, param_distributions=param_grid, cv=kfolds, n_iter=100,
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)

_ = grid_search.fit(X_train, y_train)

bestRecallTree = grid_search.best_estimator_

print(grid_search.best_params_)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


c:\Users\mukes\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1109: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


{'solver': 'adam', 'max_iter': 5000, 'learning_rate_init': 0.001, 'learning_rate': 'invscaling', 'hidden_layer_sizes': (70,), 'alpha': 1, 'activation': 'logistic'}
Wall time: 37.2 s


In [24]:
model_preds = grid_search_tree.predict(X_test)
c_matrix = confusion_matrix(y_test, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"NN-random", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,default logistic,0.826087,0.839286,0.979167,0.903846
0,rand_search_log_l2,0.827586,0.840708,0.979381,0.904762
0,grid_search_log_l1,0.827586,0.840708,0.979381,0.904762
0,lin svm,0.797980,0.814433,0.975309,0.887640
0,dtree,0.994595,0.993056,1.000000,0.996516
0,rand_poly_tree,0.989189,0.993007,0.993007,0.993007
0,grid_poly_dtree,0.994565,0.993007,1.000000,0.996491
0,"NN-60,50,40",0.994565,0.993007,1.000000,0.996491
0,NN-random,0.994565,0.993007,1.000000,0.996491


In [25]:
%%time

score_measure = "accuracy"
kfolds = 5

param_grid = {
    'hidden_layer_sizes': [ (30,), (50,), (70,), (90,)],
    'activation': ['tanh', 'relu'],
    'solver': ['adam'],
    'alpha': [.5, .7, 1],
    'learning_rate': ['adaptive', 'invscaling'],
    'learning_rate_init': [0.005, 0.01, 0.15],
    'max_iter': [5000]
}

ann = MLPClassifier()
grid_search = GridSearchCV(estimator = ann, param_grid=param_grid, cv=kfolds, 
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)

_ = grid_search.fit(X_train, y_train)

bestRecallTree = grid_search.best_estimator_

print(grid_search.best_params_)

Fitting 5 folds for each of 144 candidates, totalling 720 fits


c:\Users\mukes\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1109: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


{'activation': 'tanh', 'alpha': 1, 'hidden_layer_sizes': (30,), 'learning_rate': 'invscaling', 'learning_rate_init': 0.005, 'max_iter': 5000, 'solver': 'adam'}
Wall time: 11.4 s


In [26]:
model_preds = grid_search_tree.predict(X_test)
c_matrix = confusion_matrix(y_test, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"NN-grid", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,default logistic,0.826087,0.839286,0.979167,0.903846
0,rand_search_log_l2,0.827586,0.840708,0.979381,0.904762
0,grid_search_log_l1,0.827586,0.840708,0.979381,0.904762
0,lin svm,0.797980,0.814433,0.975309,0.887640
0,dtree,0.994595,0.993056,1.000000,0.996516
0,rand_poly_tree,0.989189,0.993007,0.993007,0.993007
0,grid_poly_dtree,0.994565,0.993007,1.000000,0.996491
0,"NN-60,50,40",0.994565,0.993007,1.000000,0.996491
0,NN-random,0.994565,0.993007,1.000000,0.996491
0,NN-grid,0.994565,0.993007,1.000000,0.996491


In [32]:
performance.sort_values(by=['Precision'])

,model,Accuracy,Precision,Recall,F1
0,lin svm,0.797980,0.814433,0.975309,0.887640
0,rand_poly_svm,0.797980,0.814433,0.975309,0.887640
0,default logistic,0.826087,0.839286,0.979167,0.903846
0,rand_search_log_l2,0.827586,0.840708,0.979381,0.904762
0,grid_search_log_l1,0.827586,0.840708,0.979381,0.904762
0,rand_poly_tree,0.989189,0.993007,0.993007,0.993007
0,grid_poly_dtree,0.994565,0.993007,1.000000,0.996491
0,"NN-60,50,40",0.994565,0.993007,1.000000,0.996491
0,NN-random,0.994565,0.993007,1.000000,0.996491
0,NN-grid,0.994565,0.993007,1.000000,0.996491


From the above results we can say that the Dtree is still the best performing model according to the precision score. However increasing the hidden layers doesnt change the classification metrics. This might be due to the overfitting of the model. The accuracy was same for the Dtree and nueral networks but the value of precision was higher for Dtree so this is the best model.